In [ ]:
import re
import string
import pandas as pd
from functools import reduce
from math import log2

In [ ]:
corpus = """
auto
car wash
auto car wash auto
machine
wash machine
""".split("\n")[1:-1]

In [ ]:
arrayText = []
for text in corpus:
    arrayText.append(text.lower().split())
arrayText

[['auto'],
 ['car', 'wash'],
 ['auto', 'car', 'wash', 'auto'],
 ['machine'],
 ['wash', 'machine']]

In [ ]:
word_set = set(arrayText[0])
for i in range(1, len(arrayText)):
    word_set = word_set.union(set(arrayText[i]))
print(word_set)

{'car', 'wash', 'auto', 'machine'}


In [ ]:
word_dicts = [None] * len(arrayText)

for i in range(len(arrayText)):
    word_dicts[i] = dict.fromkeys(word_set, 0)
    for word in arrayText[i]:
        word_dicts[i][word] += 1
        
pd.DataFrame(word_dicts)

,car,wash,auto,machine
0,0,0,1,0
1,1,1,0,0
2,1,1,2,0
3,0,0,0,1
4,0,1,0,1


In [ ]:
def compute_tf(word_dict, l):
    tf = {}
    sum_nk = len(l)
    for word, count in word_dict.items():
        tf[word] = count/sum_nk
    return tf

In [ ]:
tfs = [None] * len(arrayText)

for i in range(len(arrayText)):
    tfs[i] = compute_tf(word_dicts[i], arrayText[i])

In [ ]:
def compute_idf(strings_list):
    n = len(strings_list)
    idf = dict.fromkeys(strings_list[0].keys(), 0)
    for l in strings_list:
        for word, count in l.items():
            if count > 0:
                idf[word] += count
    
    for word, v in idf.items():
        idf[word] = log2(n / float(v))
    return idf

In [ ]:
idf = compute_idf(word_dicts)
idf

{'auto': 0.7369655941662062,
 'car': 1.3219280948873624,
 'machine': 1.3219280948873624,
 'wash': 0.7369655941662062}

In [ ]:
def compute_tf_idf(tf, idf):
    tf_idf = dict.fromkeys(tf.keys(), 0)
    for word, v in tf.items():
        tf_idf[word] = v * idf[word]
    return tf_idf

In [ ]:
tf_idfs = [None] * len(arrayText)

for i in range(len(arrayText)):
    tf_idfs[i] = compute_tf_idf(tfs[i], idf)

In [ ]:
pd.DataFrame(tf_idfs)

,car,wash,auto,machine
0,0.000000,0.000000,0.736966,0.000000
1,0.660964,0.368483,0.000000,0.000000
2,0.330482,0.184241,0.368483,0.000000
3,0.000000,0.000000,0.000000,1.321928
4,0.000000,0.368483,0.000000,0.660964
